# Code Structure Explanation

## Module Imports:
- `pandas` is used to handle CSV files as DataFrames.
- `mysql.connector` is used to connect and interact with a MySQL database.
- `os` is used for file path operations.

## CSV Files and Table Names:
- A list of tuples containing CSV file names and their corresponding table names in the MySQL database.

## Database Connection:
- Establishes a connection to the MySQL database with specified credentials (host, user, password, and database).

## Folder Path:
- The folder path where the CSV files are stored.

## Data Type Mapping Function:
- `get_sql_type()` determines the appropriate SQL data type based on the DataFrame's column data types.

## Main Loop:
- Iterates over each CSV file, reads it into a DataFrame, and replaces NaN values with `None`.
- Cleans column names to ensure they are SQL-compatible.
- Dynamically generates a `CREATE TABLE` statement based on the DataFrame's columns and data types.
- Inserts data from each row of the DataFrame into the MySQL table.

## SQL Execution:
- Executes `CREATE TABLE` and `INSERT` SQL statements for each CSV file.

## Transaction Management:
- Commits the transaction for each CSV file after processing it.

## Connection Closure:
- Closes the connection to the MySQL database.

## Notes:
- Ensure that your MySQL server is running and the credentials provided are correct.



In [ ]:
import pandas as pd
import mysql.connector
import os

In [ ]:
# List of CSV files and their corresponding table names
# This list contains tuples where each tuple holds a CSV file name and its corresponding table name in the database.
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items')  # Added payments.csv for specific handling
]

In [ ]:
# Connect to the MySQL database
# The connection details (host, user, password, and database) are specified here.
conn = mysql.connector.connect(
    host='localhost',  # Use 'localhost' for local MySQL server
    user='root',
    password='deleted',
    database='ecommerce'
)
cursor = conn.cursor()  # Create a cursor to interact with the MySQL database

In [ ]:
# The folder path where all the CSV files are stored
folder_path = 'C:\\Users\\GourangaJha\\Documents\\ETEP\\SQL_py\\ETEDAP_3\\data'

In [ ]:
# Function to determine the appropriate SQL data type based on pandas DataFrame column types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):  # If column is of integer type
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):  # If column is of float type
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):  # If column is of boolean type
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):  # If column is of datetime type
        return 'DATETIME'
    else:  # Default to TEXT for all other types
        return 'TEXT'

In [ ]:
# Loop through each CSV file and its corresponding table name
for csv_file, table_name in csv_files:
    # Create the full path to the current CSV file
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN values with None to handle SQL NULL values
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Print information about NaN values before replacement
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names by replacing spaces, hyphens, and periods with underscores
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)  # Execute the CREATE TABLE statement

    # Insert DataFrame data into the MySQL table row by row
    for _, row in df.iterrows():
        # Convert the row into a tuple, replacing NaN with None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        # Construct the INSERT statement dynamically
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)  # Execute the INSERT statement

    # Commit the transaction after processing the entire CSV file
    conn.commit()

# Close the database connection
conn.close()
